## Language Models Case: Auto Complete 

A language model assigns the probability to a sequence of words (quantitfying text). Later, the vector of probability
will be used for other tasks like machine translation or speech recognition. 

In [55]:
import math 
import random
import numpy as np
import pandas as pd
import nltk
from collections import defaultdict
from IPython.display import display
nltk.data.path.append('.')
random.seed(87)

In [3]:
with open('en_US.twitter.txt', 'r') as f:
    data = f.read()
print(type(data))
print(len(data))
display(data[:300])
display(data[-300:])

<class 'str'>
3335477


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

### Pre-process the text 

In [9]:
# split data by "\n"
def split_to_sentences(text):
    """
    Input:
        text: a series of strings 
    Output:
        a list of sentences
    """
    sentences_list = text.split("\n")
    sentences_list = [s.strip() for s in sentences_list]  # strip out the space
    sentences_list = [s for s in sentences_list if len(s) > 0]  # drop none
    return sentences_list

In [10]:
# test 
x = """
I have a pen.\n I have an apple. \nAh \n Apple pen.\n
"""
split_to_sentences(x)

['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

In [20]:
def tokenize_sentences(text_list):
    """
    Input:
        test_list: like ['I have a pen', 'apple loves']
    Output:
        a list of token (words)
    """
    token_list = []
    for text in text_list:
        text = text.lower()
        tokenized = nltk.word_tokenize(text)
        token_list.append(tokenized)
        
    return token_list

In [21]:
# test your code
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

In [22]:
def text_to_token(text):
    sentences = split_to_sentences(text)
    token = tokenize_sentences(sentences)
    
    return token

In [58]:
def sample_split(data, train_percent, validation_percent):
    """
    Input:
        data: list of sentence
        train_percent: e.g. 60%
        validation_percent: e.g. 30%
    Note: train_percent + validation_percent <= 100 
    Output:
        train_data: list of sentences 
        validation_data
        test_data
    """
    random.seed(87)
    random.shuffle(data)
    train_size = int(len(data) * train_percent /100)
    train_data = data[:train_size]
    
    validation_size = int(len(data) * validation_percent / 100)
    if train_percent + validation_percent == 100:
        validatation_data = data[train_size:]
    else:
        validatation_data = data[train_size:train_size+validation_size]
    
    test_data = data[train_size+validation_size:]
    
    return train_data, validatation_data, test_data 

In [89]:
tokenized_data = text_to_token(data)

In [90]:
# train_data, validation_data, test_data = sample_split(tokenized_data, 80, 20)
random.seed(87)
random.shuffle(tokenized_data)
train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [91]:
print(len(train_data), len(test_data))
print(train_data[0], test_data[0])

38368 9593
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production'] ['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


In [80]:
# calculate words frequency
def count_words(token_list):
    """
    Input:
        token list (WARNING: nexted list)
    Output:
        dict of words frequency 
    """
    words_freq = {}
    
    for token in token_list:
        for word in token:
            words_freq[word] = words_freq.get(word, 0) + 1
            
    return words_freq
        

In [42]:
test_token = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(test_token)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

In [74]:
# build up a bag of words with frequency > n
def hig_frequency_words(token_list, n):
    """
    Input:
        token_list: nested list of tokens
    Output:
        n: count threshold 
    """
    vocabulary = []
    word_frequency = count_words(token_list)
    for word, count in word_frequency.items():
        if count >= n:
            vocabulary.append(word)
            
    return vocabulary

In [45]:
hig_frequency_words(test_token, 2)

['.', 'are']

In [48]:
# replace out of vocabulary
def replace_oov(token_list, vocabulary, unknow_token = "<unk>"):
    vocabulary = set(vocabulary)  # set it as a set (union)
    replaced_token_list = []
    for token in token_list:
        replaced_token = []
        for word in token:
            if word in vocabulary:
                replaced_token.append(word)
            else:
                replaced_token.append(unknow_token)
        replaced_token_list.append(replaced_token)
    
    return replaced_token_list

In [49]:
vocabulary = ['sky', 'green']
replace_oov(test_token, vocabulary)

[['sky', '<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', 'green', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>']]

In [50]:
def preprocess_text(train_data, test_data, count_threshold):
    """
    preprocess data:
        - find tokens that appear at least N times
        - replace tokens that appear less than N times by "<unk>"
    Input:
        train_data
        test_data
        count_threshold
    Output:
        Tuple of:
            trainning data with low frequency words replaced by "<unk>"
            test data .......
            vocabulary of words 
    """
    vocabulary = hig_frequency_words(train_data, n=count_threshold)
    train_data_replaced = replace_oov(train_data, vocabulary)
    test_data_replaced = replace_oov(test_data, vocabulary)
    
    return train_data_replaced, test_data_replaced, vocabulary

In [51]:
tmp_train = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_text(tmp_train, 
                                                           tmp_test, 
                                                           count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


In [92]:
# preprocessing the train and test data
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_text(train_data,
                                                                        test_data, 
                                                                        minimum_freq)

In [93]:
print(train_data_processed[0], test_data_processed[0], vocabulary[0:10], 
      len(train_data_processed),
      len(test_data_processed),
      len(vocabulary))

['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production'] ['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>'] ['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the'] 38368 9593 14823


### Develop a N-gram model

In [2]:
def count_n_grams(data, n, start_token='<s>', end_token='<e>'):
    """
    Input:
        data: list of lists of words
        n: number of words in a sequence
    Output:
        a dict that maps a tuple of n-words to its frequency
    """
    n_grams_dict = {}
    for setence in data:
        setence = [start_token] * n + setence + ['<e>']
        setence = [tuple(setence[i:i+n]) for i in range(len(setence)-n+1)]
        for i in range(len(setence)):
            n_gram = setence[i]
            n_grams_dict[n_gram] = n_grams_dict.get(n_gram, 0) + 1
            
    return n_grams_dict

In [103]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


In [10]:
def estimate_probability(word, n_gram_list, n_gram_dict, n_plus1_gram_dict, vocabulary_size, k = 1.0):
    """
    Estmate the probabilities of a next word using the n-gram counts with k-smoothing
    Input:
        word: next word
        n_gram_list
        n_gram_dict: key + count
        vocabulary_list
    Output:
        a probability
    """
    previous_n_gram = tuple(n_gram_list)
    previous_n_gram_count = n_gram_dict.get(previous_n_gram, 0)
    denominator = previous_n_gram_count + k * vocabulary_size
    
    n_plus1_gram = previous_n_gram + (word, )  # do not use tuple(word) = ('w', 'o')
    
    numerator = n_plus1_gram_dict.get(n_plus1_gram, 0) + k
    
    probability = numerator / denominator
    
    return probability

In [125]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


In [25]:
def estimate_probabilities(n_gram, n_gram_dict, n_plus_gram_dict, vocabulary, k = 1.0):
    previous_n_gram = tuple(n_gram)
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    
    probabilities = {}
    for word in vocabulary:
        prob = estimate_probability(word, previous_n_gram,
                                    n_gram_dict,
                                    n_plus_gram_dict,
                                    vocabulary_size, k=k)
        probabilities[word] = prob
    
    return probabilities

In [127]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'i': 0.09090909090909091,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'this': 0.09090909090909091,
 'cat': 0.2727272727272727,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [128]:
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=1)

{'i': 0.18181818181818182,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'this': 0.18181818181818182,
 'cat': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [133]:
def make_count_matrix(n_plus1_gram_dict, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    n_grams = []
    for n_plus1_gram in n_plus1_gram_dict.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    
    row_index = {n_gram: i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_dict.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
        
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    
    return count_matrix

In [134]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))

bigram counts


,i,a,is,like,dog,this,cat,<e>,<unk>
"(like,)",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(<s>,)",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(dog,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(i,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(is,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [135]:
# Show trigram counts
print('\ntrigram counts')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))


trigram counts


,i,a,is,like,dog,this,cat,<e>,<unk>
"(like, a)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(dog, is)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(this, dog)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, <s>)",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(is, like)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, i)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(<s>, this)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [136]:
# calcualte probabilities
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k  # smoothing 
    # vertically downwards across rows (axis 0)
    # # running horizontally across columns (axis 1).
    row_sum = count_matrix.sum(axis=1)  # sum of row
    prob_matrix = count_matrix.div(row_sum, axis=0)
    return prob_matrix

In [137]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k=1))

bigram probabilities


,i,a,is,like,dog,this,cat,<e>,<unk>
"(like,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(<s>,)",0.181818,0.090909,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.090909
"(dog,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(this,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(i,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909
"(is,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000


In [19]:
def calculate_perplexity(sentence, n_gram_dict, n_plus1_dict, vocabulary_size, k=1.0):
    """
    Calculate perplexity for one sentence
    """
    n = len(list(n_gram_dict.keys())[0])  # n = len of grams not list 
    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    
    N = len(sentence)
    
    product_pi = 1.0  # initialize the pi
    for t in range(n, N):
        n_gram = sentence[t-n:t]  # t-n to t 
        word = sentence[t]
        
        prob = estimate_probability(word, n_gram, n_gram_dict, n_plus1_dict,
                                    vocabulary_size, k=k)
        product_pi *= 1/prob
    
    perplexity = product_pi**(1/N)
    
    return perplexity

In [20]:
# test your code

sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


In [22]:
def suggest_a_word(previous_token, n_gram_dict, n_plus1_gram_dict, vocabulary, k=1.0, start_with=None):
    n = len(list(n_gram_dict.keys())[0])
    # previous token can be shorter or longer than n_gram
    previous_n_gram = previous_token[-n:]
    
    probabilities = estimate_probabilities(previous_n_gram, n_gram_dict, n_plus1_gram_dict,
                                           vocabulary, k=k)
    suggestion = None
    
    max_prob = 0
    
    for word, prob in probabilities.items():
        # following up word starting with 'c' or something else
        if start_with != None:
            if not word.startswith(start_with):
                continue
            
        if prob > max_prob:
            suggestion = word
            max_prob = prob
            
    return suggestion, max_prob
            
        

In [27]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}",
      "when we specifying the staring character")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like',
	and the suggested word is `cat` with a probability of 0.0909 when we specifying the staring character


In [28]:
# suggest based on several models
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [29]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('is', 0.1111111111111111),
 ('is', 0.1111111111111111)]